# DER Scenario Planning with Monte Carlo Simulation

From the [Sisyphean Gridworks ML Playground](https://sgridworks.com/ml-playground/guides/07-der-scenario-planning.html)

## Setup

Clone the repository and install dependencies. Run this cell first.

In [ ]:
!git clone https://github.com/SGridworks/Dynamic-Network-Model.git 2>/dev/null || echo 'Already cloned'
%cd Dynamic-Network-Model
!pip install -q pandas numpy matplotlib seaborn scikit-learn xgboost lightgbm pyarrow

## Load Scenario Configurations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from demo_data.load_demo_data import (
    load_growth_scenarios, load_load_profiles,
    load_solar_installations, load_ev_chargers
)

# Load the four scenario projections (indexed by scenario_id, year)
# Columns: scenario_name, ev_adoption_pct, solar_adoption_pct,
#          battery_adoption_pct, load_growth_pct, etc.
scenarios = load_growth_scenarios()
print(scenarios.head(10))
print(f"\nScenario names: {scenarios.index.get_level_values('scenario_id').unique().tolist()}")

## Load Feeder Capacity Data

In [ ]:
# Load feeder-level 15-minute load profiles
from demo_data.load_demo_data import load_customers
load = load_load_profiles()

# Calculate peak load and average load per feeder
feeder_summary = load.groupby("feeder_id").agg(
    peak_mw=("load_mw", "max"),
    avg_mw=("load_mw", "mean")
).reset_index()

# Get customer counts from the customers table
customers = load_customers()
customer_counts = customers.groupby("feeder_id").size().reset_index(name="customer_count")
feeder_summary = feeder_summary.merge(customer_counts, on="feeder_id", how="left")

# Assume rated capacity is 20% above historical peak
feeder_summary["rated_capacity_mw"] = feeder_summary["peak_mw"] * 1.2
feeder_summary["headroom_mw"] = (feeder_summary["rated_capacity_mw"]
                                  - feeder_summary["peak_mw"])

print(feeder_summary.to_string(index=False))

## Define the Monte Carlo Parameters

In [ ]:
# Uncertain parameters and their ranges
# Solar: 15-40% penetration by 2030 (uniform distribution)
# EV: 8-25% penetration by 2030 (uniform distribution)
# Average solar system size: 5-10 kW
# Average EV charger load: 6-12 kW (Level 2)

N_SIMULATIONS = 1000
N_FEEDERS = 65
TOTAL_CUSTOMERS = 140000

# Store results
results = []

np.random.seed(42)  # for reproducibility

for sim in range(N_SIMULATIONS):
    # Draw random adoption rates
    solar_pct  = np.random.uniform(0.15, 0.40)
    ev_pct     = np.random.uniform(0.08, 0.25)
    solar_kw   = np.random.uniform(5, 10)
    ev_kw      = np.random.uniform(6, 12)

    # Distribute adoptions across feeders (proportional to customers)
    for _, feeder in feeder_summary.iterrows():
        fid = feeder["feeder_id"]
        customers = feeder["customer_count"]

        # Number of new solar and EV installations on this feeder
        n_solar = int(customers * solar_pct)
        n_ev    = int(customers * ev_pct)

        # Peak impact on the feeder (MW)
        # Solar reduces net load during day, EV adds load during evening
        solar_mw = (n_solar * solar_kw) / 1000
        ev_mw    = (n_ev * ev_kw) / 1000

        # Net peak impact: EV adds, solar doesn't help at evening peak
        new_peak = feeder["peak_mw"] + ev_mw
        exceeds_capacity = new_peak > feeder["rated_capacity_mw"]

        results.append({
            "sim": sim, "feeder_id": fid,
            "solar_pct": solar_pct, "ev_pct": ev_pct,
            "new_peak_mw": new_peak,
            "rated_capacity_mw": feeder["rated_capacity_mw"],
            "exceeds_capacity": exceeds_capacity,
            "solar_generation_mw": solar_mw,
            "ev_load_mw": ev_mw
        })

results_df = pd.DataFrame(results)
print(f"Total simulation runs: {len(results_df):,}")

## Analyze Results: Which Feeders Are at Risk?

In [ ]:
# Probability of exceeding capacity per feeder
risk_by_feeder = results_df.groupby("feeder_id")["exceeds_capacity"].mean()
risk_by_feeder = risk_by_feeder.sort_values(ascending=False)

print("Probability of exceeding capacity by 2030:\n")
for fid, prob in risk_by_feeder.items():
    bar = "#" * int(prob * 50)
    print(f"  {fid}: {prob:>5.1%}  {bar}")

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
risk_by_feeder.plot(kind="bar", color="#5FCCDB", ax=ax)
ax.axhline(y=0.5, color="red", linestyle="--", label="50% probability threshold")
ax.set_ylabel("Probability of Overload")
ax.set_title("Feeder Overload Risk by 2030 (Monte Carlo)")
ax.legend()
plt.tight_layout()
plt.show()

## Visualize the Uncertainty Distribution

In [ ]:
# For the highest-risk feeder, show the distribution of peak loads
highest_risk_feeder = risk_by_feeder.idxmax()
feeder_results = results_df[results_df["feeder_id"] == highest_risk_feeder]
rated_cap = feeder_results["rated_capacity_mw"].iloc[0]

fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(feeder_results["new_peak_mw"], bins=50, color="#5FCCDB",
       edgecolor="white", alpha=0.8)
ax.axvline(x=rated_cap, color="red", linewidth=2, linestyle="--",
           label=f"Rated Capacity ({rated_cap:.1f} MW)")
ax.set_xlabel("Projected Peak Load (MW)")
ax.set_ylabel("Number of Simulations")
ax.set_title(f"Peak Load Distribution — Feeder {highest_risk_feeder} (2030)")
ax.legend()
plt.tight_layout()
plt.show()

## Compare Scenarios

In [ ]:
# Run four deterministic scenarios for comparison
# Pull 2030 adoption rates from the scenarios DataFrame loaded in Step 1
scenario_params = {}
for scn_id in scenarios.index.get_level_values("scenario_id").unique():
    row = scenarios.loc[(scn_id, 2030)]
    scenario_params[row["scenario_name"]] = {
        "solar_pct": row["solar_adoption_pct"] / 100,
        "ev_pct": row["ev_adoption_pct"] / 100,
    }

fig, axes = plt.subplots(1, 4, figsize=(18, 5), sharey=True)

for ax, (name, params) in zip(axes, scenario_params.items()):
    peaks = []
    for _, feeder in feeder_summary.iterrows():
        ev_mw = (feeder["customer_count"] * params["ev_pct"] * 8) / 1000
        new_peak = feeder["peak_mw"] + ev_mw
        peaks.append({
            "feeder": feeder["feeder_id"],
            "peak": new_peak,
            "capacity": feeder["rated_capacity_mw"]
        })
    pdf = pd.DataFrame(peaks)

    colors = ["#fc8181" if r["peak"] > r["capacity"] else "#5FCCDB"
              for _, r in pdf.iterrows()]
    ax.bar(pdf["feeder"], pdf["peak"], color=colors)
    ax.set_title(name, fontsize=10)
    ax.set_xlabel("Feeder")
    if ax == axes[0]:
        ax.set_ylabel("Peak Load (MW)")
    ax.tick_params(axis="x", rotation=45)

plt.suptitle("Feeder Peak Loads Under Different DER Scenarios (2030)", fontsize=14)
plt.tight_layout()
plt.show()

## What You Built and Next Steps